<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_06_validacion_y_modelos_clasicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesion 06 — Validación y Modelos Clásicos


# Parte A - Validación cruzada y métricas de performance

---


## Objetivos
- Entender la importancia de la validación cruzada.
- Conocer métricas comunes: Accuracy, Precision, Recall, F1, ROC-AUC.
- Evaluar un modelo de churn de forma rigurosa.

---

## 1. Validación Cruzada
Divide los datos en **k-folds**.  
Cada fold se usa una vez para test y el resto para train.  
Ventaja: evita depender de una sola partición.

*Referencia:* https://www.datacamp.com/es/tutorial/k-fold-cross-validation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# Parámetros
n_samples = 25
n_splits = 5
indices = np.arange(n_samples)

# Colores para las 5 carpetas
colors = plt.cm.tab10(np.linspace(0, 1, n_splits))

# Crear figura
fig, ax = plt.subplots(figsize=(10, 2))
bars = ax.bar(indices, np.ones_like(indices), color="lightgray", edgecolor="black")

ax.set_xticks(indices)
ax.set_yticks([])
ax.set_title("Validación Cruzada de 5 Carpetas", fontsize=14)
ax.set_xlabel("Observaciones")

def update(frame):
    for bar in bars:
        bar.set_color("lightgray")
    test_idx = np.array_split(indices, n_splits)[frame]
    for i in range(len(indices)):
        if i in test_idx:
            bars[i].set_color(colors[frame])  # Test set
        else:
            bars[i].set_color("lightblue")   # Train set
    ax.set_title(f"Validación Cruzada - Fold {frame+1}", fontsize=14)
    return bars

ani = animation.FuncAnimation(fig, update, frames=n_splits, blit=False, repeat=True)

# Guardar como GIF sin mostrar
gif_path = "cross_validation_5fold.gif"
ani.save(gif_path, writer="pillow", fps=1)
plt.close(fig)  # Cierra la figura para que no se renderice en Jupyter

#print(f"Animación guardada en: {gif_path}")

![Animación de Validación Cruzada](cross_validation_5fold.gif)

## 2. Ejemplo con Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 1. Cargar dataset
url = "https://raw.githubusercontent.com/Geo-y20/Telco-Customer-Churn-/refs/heads/main/Telco%20Customer%20Churn.csv"
df = pd.read_csv(url)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["TotalCharges"] = df["TotalCharges"].fillna(df["MonthlyCharges"])
display(df.head())

# 2. Eliminar columnas irrelevantes
df = df.drop("customerID", axis=1)

# 3. One-Hot Encoding con pandas para variables categóricas
df = pd.get_dummies(df, drop_first=True)

# 4. Separar variables y target (Definir X e y)
X = df.drop("Churn_Yes", axis=1)  # porque get_dummies crea Churn_No y Churn_Yes
y = df["Churn_Yes"]

# 5. Escalar datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 6. Definir modelo
clf = LogisticRegression(max_iter=5000)

# 7. Validación cruzada
scores = cross_val_score(clf, X, y, cv=5, scoring="accuracy")
print("Accuracy promedio:", np.mean(scores))


---

## 3. Métricas de Clasificación

* **Accuracy**: % de predicciones correctas.
* **Precision**: proporción de positivos predichos que son correctos.
* **Recall**: proporción de verdaderos positivos detectados.
* **F1**: balance entre precision y recall.
* **ROC-AUC**: mide discriminación entre clases.


---

## 4. Curva ROC

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

clf.fit(X_train, y_train)

RocCurveDisplay.from_estimator(clf, X_test, y_test)
plt.show()

---

## 5. Ejercicio Guiado

1. Aplicar validación cruzada al modelo de churn.
2. Calcular precision, recall y f1-score.
3. Graficar la curva ROC.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

# 1) Cargar dataset
url = "https://raw.githubusercontent.com/Geo-y20/Telco-Customer-Churn-/refs/heads/main/Telco%20Customer%20Churn.csv"
df = pd.read_csv(url)

# Limpieza mínima recomendada por este dataset
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["TotalCharges"] = df["TotalCharges"].fillna(df["MonthlyCharges"])

# 2) Eliminar columnas irrelevantes
df = df.drop(columns=["customerID"])

# 3) One-Hot Encoding
df = pd.get_dummies(df, drop_first=True)

# 4) X, y
X = df.drop(columns=["Churn_Yes"])
y = df["Churn_Yes"]

# 5) Modelo en Pipeline para evitar fuga de información del escalado
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])

# 6) Definir validación cruzada y métricas
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

# 7) cross_validate para obtener métricas por fold y promedio
cv_results = cross_validate(pipe, X, y, cv=cv, scoring=scoring, return_train_score=False)

print("=== Métricas promedio (5-fold CV) ===")
for m in scoring.keys():
    vals = cv_results[f"test_{m}"]
    print(f"{m.capitalize():<9}: {vals.mean():.4f}  (± {vals.std():.4f})")

# 8) Predicciones out-of-fold para reportes agregados (confusion matrix, ROC, etc.)
#    cross_val_predict entrena en cada fold y predice sobre el holdout de ese fold.
y_pred_oof = cross_val_predict(pipe, X, y, cv=cv, method="predict")
y_prob_oof = cross_val_predict(pipe, X, y, cv=cv, method="predict_proba")[:, 1]

# 9) Reportes agregados con OOF
print("\n=== Reporte de clasificación (OOF) ===")
print(classification_report(y, y_pred_oof, digits=4))

cm = confusion_matrix(y, y_pred_oof)
print("=== Matriz de confusión (OOF) ===")
print(cm)

acc  = accuracy_score(y, y_pred_oof)
prec = precision_score(y, y_pred_oof)
rec  = recall_score(y, y_pred_oof)
f1   = f1_score(y, y_pred_oof)
rocA = roc_auc_score(y, y_prob_oof)

print("\n=== Métricas agregadas (OOF) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1       : {f1:.4f}")
print(f"ROC-AUC  : {rocA:.4f}")

# 10) Curva ROC (OOF)
fpr, tpr, _ = roc_curve(y, y_prob_oof)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {rocA:.3f})")
plt.plot([0, 1], [0, 1], linestyle="--", label="Azar")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC (validación cruzada OOF)")
plt.legend(loc="lower right")
plt.show()


---

## 6. Preguntas de Discusión

### 6.1. ¿Por qué conviene usar validación cruzada?

* **Idea central:** La validación cruzada permite evaluar el desempeño de un modelo no en un único “corte” de train/test, sino en múltiples particiones.
* **Beneficio:**

  * Reduce la **varianza** en la estimación del desempeño.
  * Asegura que todos los datos participan alguna vez como entrenamiento y como validación.
  * Evita que nuestra evaluación dependa de un “golpe de suerte” en cómo partimos los datos.
* **Analogía:** Es como probar un coche no solo en una pista lisa, sino también en caminos distintos para asegurarnos de que rinde bien en general.

---

### 6.2. ¿Qué métrica elegir si me importa más detectar todos los clientes que van a fugarse (churn)?

* Aquí hablamos de **sensibilidad/recall**.
* **Recall** = de todos los que realmente se fueron, ¿cuántos detecté?
* Si tu objetivo es **no perder a ningún cliente que se fuga**, necesitas maximizar el recall, incluso si eso significa aceptar más falsos positivos.
* **Ejemplo:** Es como un detector de incendios: mejor que suene muchas veces aunque sea por vapor de la ducha, a que no suene cuando hay fuego real.

---

### 6.3. ¿Y si el costo de un falso positivo es muy alto?

* Entonces priorizamos la **precisión**.
* **Precisión** = de todos los que marqué como fuga, ¿cuántos realmente se fueron?
* Si contactar o dar beneficios a un cliente mal clasificado cuesta mucho, entonces conviene sacrificar recall y ganar en precisión.
* **Ejemplo:** Campañas de retención costosas: no quieres gastar en clientes que igual no se iban a ir.

---

### 6.4. ¿Por qué ROC-AUC es preferible a Accuracy en problemas desbalanceados?

* En datasets desbalanceados, la **accuracy** puede ser engañosa:

  * Si el 95% de clientes no se fuga, un modelo que siempre prediga “no fuga” tendrá 95% de accuracy.
  * Pero ese modelo no sirve para detectar churn.
* **ROC-AUC** mide la capacidad del modelo de rankear correctamente entre positivos y negativos.

  * Es independiente del umbral de decisión.
  * Permite comparar modelos incluso en situaciones de gran desbalance.
* **Ejemplo:** En un aeropuerto, casi todos los equipajes son normales; accuracy te diría “estás perfecto” si nunca detectas nada sospechoso. Pero lo que interesa es tu habilidad de distinguir maletas peligrosas de las normales.

### 6.5. ¿Qué trade-off aceptarías en un banco? ¿Y en un hospital?

| Contexto                   | Costo más grave                                | Métrica a priorizar           | Ejemplo de decisión                                                                 |
| -------------------------- | ---------------------------------------------- | ----------------------------- | ----------------------------------------------------------------------------------- |
| **Banco – Churn**          | Falso negativo (perder un cliente que se fuga) | Recall (Sensibilidad)         | Mejor contactar de más que perder clientes valiosos                                 |
| **Banco – Fraude**         | Falso positivo (bloquear cliente inocente)     | Precisión                     | Minimizar molestias y costos reputacionales                                         |
| **Banco – Scoring (tradicional)** | Falso positivo (dar crédito a quien no paga)   | Precisión en clase “aprobado” | Mejor rechazar a algunos buenos que aprobar demasiados malos                        |
| **Banco – Scoring (inclusión/fintech)** | Falso negativo (rechazar a un cliente confiable) | Recall (Sensibilidad)         | Mejor aceptar a casi todos los buenos clientes aunque se filtren algunos malos      |
| **Hospital – Diagnóstico** | Falso negativo (no detectar enfermo)           | Recall (Sensibilidad)         | Mejor hacer pruebas extras que dejar un enfermo sin diagnóstico                     |


---


# Parte B: Modelos clásicos (Logistic Regression y Árboles)

---


## Objetivos
- Aplicar Logistic Regression a un caso real de churn.
- Comparar con un Árbol de Decisión.
- Analizar ventajas/desventajas de cada modelo.

---

## 1. Dataset Churn

---

## 2. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

lr = LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:,1]

print("Logistic Regression")
print(classification_report(y_test, y_pred_lr))
print("AUC:", roc_auc_score(y_test, y_proba_lr))

---

## 3. Árbol de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)
y_proba_tree = tree.predict_proba(X_test)[:,1]

print("Decision Tree")
print(classification_report(y_test, y_pred_tree))
print("AUC:", roc_auc_score(y_test, y_proba_tree))

---

## 4. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Definir modelo Random Forest
rf = RandomForestClassifier(
    n_estimators=200,   # número de árboles
    max_depth=6,        # profundidad máxima de cada árbol
    random_state=42
)

# Entrenar
rf.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

# Métricas
print("Random Forest")
print(classification_report(y_test, y_pred_rf))
print("AUC:", roc_auc_score(y_test, y_proba_rf))


---

🔎 **Diferencias clave RandomForestClassifier vs DecisionTreeClassifier**:

* `RandomForestClassifier` combina muchos árboles (bagging) → más robusto y menos sobreajuste.
* Hiperparámetros importantes:

  * `n_estimators`: nº de árboles (100–500 es común).
  * `max_depth`: controla complejidad (igual que en árboles individuales).
  * `max_features`: nº de variables usadas en cada split (default = sqrt para clasificación).
* Se obtiene `predict` y `predict_proba` igual que en el árbol simple.


---

## 4. Comparación Gráfica

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

RocCurveDisplay.from_estimator(lr, X_test, y_test, name="Logistic Regression")
RocCurveDisplay.from_estimator(tree, X_test, y_test, name="Decision Tree")
RocCurveDisplay.from_estimator(rf, X_test, y_test, name="Random Forest")
plt.show()

---

## 5. Discusión

* **Logistic Regression**:

  * Modelo lineal, interpretable.
  * Útil como *baseline* porque establece un punto de referencia claro.
  * Bueno cuando las relaciones entre variables y el objetivo son aproximadamente lineales.
  * Fácil de explicar a negocio (“cada variable aumenta/disminuye la probabilidad de churn en tanto %”).

* **Decision Tree**:

  * Captura relaciones **no lineales** y **interacciones** entre variables sin necesidad de transformaciones.
  * Fácil de explicar visualmente (se puede graficar el árbol y mostrar las reglas).
  * Puede sobreajustar si no se controla la profundidad u otros parámetros.
  * Bueno para comunicación, pero menos estable (pequeños cambios en los datos → árbol diferente).

* **Random Forest**:

  * Conjunto de muchos árboles → más robusto y con mejor desempeño promedio que un solo árbol.
  * Reduce el riesgo de sobreajuste combinando resultados (bagging).
  * Captura no linealidades y complejas interacciones de forma más estable.
  * Menos interpretable que un árbol único o regresión logística (aunque se pueden usar **importancias de variables** o **SHAP** para interpretarlo).
  * Normalmente alcanza un **mejor equilibrio entre bias y varianza**.

---

## 6. Preguntas de Discusión

1. ¿Qué modelo recomendarían para presentar a un equipo de negocio?
2. ¿Qué riesgos hay de sobreajuste en un árbol?
3. ¿Cómo se podrían combinar ambos enfoques?

## 6. Preguntas de Discusión

**1. ¿Qué modelo recomendarían para presentar a un equipo de negocio?**

* **Regresión Logística** es normalmente la opción más adecuada para presentaciones a negocio porque:

  * Es **fácil de interpretar**: cada variable tiene un coeficiente que indica su impacto en la probabilidad de churn.
  * Permite explicar “qué factores incrementan la probabilidad de fuga”.
  * Ideal como **baseline** antes de pasar a modelos más complejos.
* **Árbol de Decisión** también es una buena alternativa si se quiere mostrar reglas claras y visuales (“si el cliente tiene contrato mensual y bajo tenure → alta probabilidad de churn”).

---

**2. ¿Qué riesgos hay de sobreajuste en un árbol?**

* Un **árbol profundo** puede aprender demasiado los datos de entrenamiento, detectando patrones espurios.
* Esto se traduce en **alta accuracy en train pero bajo desempeño en test**.
* Factores de riesgo:

  * No limitar la profundidad (`max_depth`).
  * No restringir el número mínimo de muestras por hoja (`min_samples_leaf`).
  * Dataset con ruido o muchas variables categóricas.

---

**3. ¿Cómo se podrían combinar ambos enfoques?**

* **Random Forest** es justamente la combinación de muchos árboles (bagging), reduciendo la varianza y mejorando la generalización.
* Otra alternativa es **Ensemble Híbrido**:

  * Usar **Regresión Logística** como baseline por su interpretabilidad.
  * Usar **Random Forest / Gradient Boosting** para mejorar predicciones.
  * Comparar ambos y reportar al negocio insights del modelo interpretable + performance del modelo robusto.
* Incluso se pueden usar **modelos en cascada**: primero un modelo interpretable para filtrar casos “claros” y luego un modelo complejo para los “dudosos”.

---
